In [ ]:
import sys, time, os, asyncio, glob, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
start = Time("2023-01-01 00:00:00Z", scale='utc')
end = start + TimeDelta(86400, format='sec')
finish = Time("2023-09-11 00:00:00Z", scale='utc')

dates = []
num_estops = []
months = []
month_labels = []
while start.unix_tai < finish.unix_tai:
    eStop = await client.select_time_series('lsst.sal.ATDome.logevent_emergencyStop', ['*'], start, end)
    gis = await client.select_time_series('lsst.sal.GIS.logevent_heartbeat', ['heartbeat'], start, end)
    print(start.isot, len(eStop), len(gis))
    dates.append(start.isot)
    if int(re.split('-|T|:',start.isot)[2]) == 1:
        months.append(start.isot)
        month_labels.append(Time(start, format='iso', out_subfmt='date').iso)
    num_estops.append(len(eStop))
    start = start + TimeDelta(86400, format='sec')
    end = start + TimeDelta(86400, format='sec')

fig, axs = plt.subplots(2,1, figsize=(10,10))
plt.subplots_adjust(hspace=0.5)
plt.suptitle("AuxTel EStop events - 2023", fontsize=24)
axs[0].plot(dates, num_estops)
axs[0].set_xticks(months)
axs[0].set_xticklabels(month_labels, rotation=45)
axs[0].set_ylabel("Number of Estop events/day")

axs[1].plot(dates, num_estops)
axs[1].set_xlim(months[-2], finish.isot)
axs[1].set_xticks(months[7:9])
axs[1].set_xticklabels(month_labels[7:9], rotation=45)
axs[1].set_ylabel("Number of Estop events/day")
plt.savefig(f"/home/craiglagegit/DATA/EStop_Issue_History_11Sep23.png")